In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import absolute_import
import argparse
import numpy as np
import torch
import os
import sys
import logging
import pdb

sys.path.insert(0,'/home/ytaille/AttentionSegmentation')

from allennlp.data import Vocabulary
from allennlp.data.iterators import DataIterator
# import allennlp.data.dataset_readers as Readers
import AttentionSegmentation.reader as Readers

# import model as Models
import AttentionSegmentation.model.classifiers as Models

from AttentionSegmentation.commons.utils import \
    setup_output_dir, read_from_config_file
from AttentionSegmentation.commons.model_utils import \
    construct_vocab, load_model_from_existing
# from AttentionSegmentation.visualization.visualize_attns import \
#     html_visualizer
import AttentionSegmentation.model.attn2labels as SegmentationModels

"""The main entry point

This is the main entry point for training HAN SOLO models.

Usage::

    ${PYTHONPATH} -m AttentionSegmentation/main
        --config_file ${CONFIG_FILE}

"""
args = type('MyClass', (object,), {'content':{}})()
args.config_file = 'Configs/config_ncbi.json'
args.log = 'INFO'
args.loglevel = 'INFO'
args.seed = 1

# Setup Experiment Directory
config = read_from_config_file(args.config_file)
if args.seed > 0:
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if config.get('trainer', None) is not None and \
       config.get('trainer', None).get('cuda_device', -1) > 0:
        torch.cuda.manual_seed(args.seed)
serial_dir, config = setup_output_dir(config, args.loglevel)
logger = logging.getLogger(__name__)

# Load Training Data
TRAIN_PATH = config.pop("train_data_path")
logger.info("Loading Training Data from {0}".format(TRAIN_PATH))
dataset_reader_params = config.pop("dataset_reader")
reader_type = dataset_reader_params.pop("type", None)
assert reader_type is not None and hasattr(Readers, reader_type),\
    f"Cannot find reader {reader_type}"
reader = getattr(Readers, reader_type).from_params(dataset_reader_params)
instances_train = reader.read(file_path=TRAIN_PATH)
instances_train = instances_train
logger.info("Length of {0}: {1}".format(
    "Training Data", len(instances_train)))

# Load Validation Data
VAL_PATH = config.pop("validation_data_path")
logger.info("Loading Validation Data from {0}".format(VAL_PATH))
instances_val = reader.read(VAL_PATH)
instances_val = instances_val
logger.info("Length of {0}: {1}".format(
    "Validation Data", len(instances_val)))

# Load Test Data
TEST_PATH = config.pop("test_data_path", None)
instances_test = None
if TEST_PATH is not None:
    logger.info("Loading Test Data from {0}".format(TEST_PATH))
    instances_test = reader.read(TEST_PATH)
    instances_test = instances_test
    logger.info("Length of {0}: {1}".format(
        "Testing Data", len(instances_test)))

# # Load Pretrained Existing Model
# load_config = config.pop("load_from", None)

# # Construct Vocabulary
vocab_size = config.pop("max_vocab_size", -1)
logger.info("Constructing Vocab of size: {0}".format(vocab_size))
vocab_size = None if vocab_size == -1 else vocab_size
vocab = Vocabulary.from_instances(instances_train,
                                  max_vocab_size=vocab_size)
vocab_dir = os.path.join(serial_dir, "vocab")
assert os.path.exists(vocab_dir), "Couldn't find the vocab directory"
vocab.save_to_files(vocab_dir)

# if load_config is not None:
#     # modify the vocab from the source model vocab
#     src_vocab_path = load_config.pop("vocab_path", None)
#     if src_vocab_path is not None:
#         vocab = construct_vocab(src_vocab_path, vocab_dir)
#         # Delete the old vocab
#         for file in os.listdir(vocab_dir):
#             os.remove(os.path.join(vocab_dir, file))
#         # save the new vocab
#         vocab.save_to_files(vocab_dir)
logger.info("Saving vocab to {0}".format(vocab_dir))
logger.info("Vocab Construction Done")

# # Construct the data iterators
logger.info("Constructing Data Iterators")
data_iterator = DataIterator.from_params(config.pop("iterator"))
data_iterator.index_with(vocab)

logger.info("Data Iterators Done")

# Create the model
logger.info("Constructing The model")
model_params = config.pop("model")
model_type = model_params.pop("type")
assert model_type is not None and hasattr(Models, model_type),\
    f"Cannot find reader {model_type}"
model = getattr(Models, model_type).from_params(
    vocab=vocab,
    params=model_params,
    label_indexer=reader.get_label_indexer()
)
logger.info("Model Construction done")

# visualize = config.pop("visualize", False)
# visualizer = None
# if visualize:
#     visualizer = html_visualizer(vocab, reader)
segmenter_params = config.pop("segmentation")
segment_class = segmenter_params.pop("type")
segmenter = getattr(SegmentationModels, segment_class).from_params(
    vocab=vocab,
    reader=reader,
    params=segmenter_params
)

# logger.info("Segmenter Done")

# print("##################################\nAYYYYYYYYYYYYYYYYYYYYYYYY\n\n\n\n\n\n\n\n###########################")

# exit()


# if load_config is not None:
#     # Load the weights, as specified by the load_config
#     model_path = load_config.pop("model_path", None)
#     layers = load_config.pop("layers", None)
#     load_config.assert_empty("Load Config")
#     assert model_path is not None,\
#         "You need to specify model path to load from"
#     model = load_model_from_existing(model_path, model, layers)
#     logger.info("Pretrained weights loaded")

# logger.info("Starting the training process")



/home/ytaille/.conda/envs/deep_multilingual_normalization/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


2020-12-14 11:32:46,262: INFO: train_data_path = /home/ytaille/data/resources/medic/ncbi_conll_ner_train.conll
2020-12-14 11:32:46,265: INFO: Loading Training Data from /home/ytaille/data/resources/medic/ncbi_conll_ner_train.conll
2020-12-14 11:32:46,268: INFO: dataset_reader.type = WeakConll2003DatasetReader
2020-12-14 11:32:46,271: INFO: dataset_reader.token_indexers.bert.type = bert-pretrained
2020-12-14 11:32:46,272: INFO: dataset_reader.token_indexers.bert.pretrained_model = ./Data/embeddings/bert-base-multilingual-cased-vocab.txt
2020-12-14 11:32:46,274: INFO: dataset_reader.token_indexers.bert.use_starting_offsets = True
2020-12-14 11:32:46,275: INFO: dataset_reader.token_indexers.bert.do_lowercase = False
2020-12-14 11:32:46,276: INFO: dataset_reader.token_indexers.bert.never_lowercase = None
2020-12-14 11:32:46,279: INFO: dataset_reader.token_indexers.bert.max_pieces = 512
2020-12-14 11:32:46,280: INFO: loading vocabulary file ./Data/embeddings/bert-base-multilingual-cased-voc

0it [00:00, ?it/s]

2020-12-14 11:32:46,429: INFO: Reading instances from lines in file at: /home/ytaille/data/resources/medic/ncbi_conll_ner_train.conll


1803it [00:01, 1699.26it/s]

...

2020-12-14 11:32:47,932: INFO: Length of Testing Data: 316
2020-12-14 11:32:47,934: INFO: max_vocab_size = -1
2020-12-14 11:32:47,935: INFO: Constructing Vocab of size: -1
2020-12-14 11:32:47,936: INFO: Fitting token dictionary from dataset.



100%|██████████| 1803/1803 [00:00<00:00, 36978.16it/s]

2020-12-14 11:32:48,000: INFO: Saving vocab to ./trained_models/NCBI-BERT-realFT-PS/run-13/vocab
2020-12-14 11:32:48,001: INFO: Vocab Construction Done
2020-12-14 11:32:48,002: INFO: Constructing Data Iterators
2020-12-14 11:32:48,003: INFO: iterator.type = bucket
2020-12-14 11:32:48,004: INFO: iterator.sorting_keys = [['tokens', 'bert']]
2020-12-14 11:32:48,005: INFO: iterator.padding_noise = 0.1
2020-12-14 11:32:48,006: INFO: iterator.biggest_batch_first = False
2020-12-14 11:32:48,010: INFO: iterator.batch_size = 32
2020-12-14 11:32:48,011: INFO: iterator.instances_per_epoch = None
2020-12-14 11:32:48,012: INFO: iterator.max_instances_in_memory = None
2020-12-14 11:32:48,013: INFO: Data Iterators Done
2020-12-14 11:32:48,013: INFO: Constructing The model
2020-12-14 11:32:48,015: INFO: model.type = MultiClassifier
2020-12-14 11:32:48,017: INFO: model.method = binary
2020-12-14 11:32:48,018: INFO: model.text_field_embedder.type = basic
2020-12-14 11:32:48,019: INFO: model.text_field_e

2020-12-14 11:32:53,131: INFO: Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 119547
}

2020-12-14 11:32:56,647: INFO: Layer encoder.layer.10.attention.self.query.weight is finetuned
2020-12-14 11:32:56,650: INFO: Layer encoder.layer.10.attention.self.query.bias is finetuned
2020-12-14 11:32:56,651: INFO: Layer encoder.layer.10.attention.self.key.weight is finetuned
2020-12-14 11:32:56,652: INFO: Layer encoder.layer.10.attention.self.key.bias is finetuned
2020-12-14 11:32:56,653: INFO: Layer encoder.layer.10.attention.self.value.wei

/home/ytaille/.conda/envs/deep_multilingual_normalization/lib/python3.6/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [3]:
data_iterator.vocab.add_token_to_namespace("@@UNKNOWN@@", "chunk_tags")

1141

In [4]:
from AttentionSegmentation.trainer import Trainer

trainer = Trainer.from_params(
    model=model,
    base_dir=serial_dir,
    iterator=data_iterator,
    train_data=instances_train,
    validation_data=instances_val,
    segmenter=segmenter,
    params=config.pop("trainer")
)


2020-12-14 11:32:57,526: INFO: PyTorch version 1.5.1 available.
2020-12-14 11:33:00,409: INFO: TensorFlow version 2.3.1 available.
2020-12-14 11:33:00,929: INFO: Loading faiss with AVX2 support.
2020-12-14 11:33:00,932: INFO: Loading faiss.
2020-12-14 11:33:01,422: INFO: trainer.patience = 10
2020-12-14 11:33:01,423: INFO: trainer.validation_metric = +accuracy
2020-12-14 11:33:01,424: INFO: trainer.num_epochs = 50
2020-12-14 11:33:01,424: INFO: trainer.cuda_device = 0
2020-12-14 11:33:01,425: INFO: trainer.grad_norm = None
2020-12-14 11:33:01,426: INFO: trainer.grad_clipping = None
2020-12-14 11:33:01,427: INFO: trainer.num_serialized_models_to_keep = 1
2020-12-14 11:33:04,148: INFO: trainer.optimizer.type = adam
2020-12-14 11:33:04,150: INFO: trainer.optimizer.parameter_groups = [[['.*bert.*'], ConfigTree([('lr', 2e-07)])], [['.*encoder_word.*', '.*attn.*', '.*logit.*'], ConfigTree([('lr', 0.001)])]]
2020-12-14 11:33:04,151: INFO: Converting Params object to dict; logging of default v

In [5]:
from __future__ import absolute_import
import logging
import os
import shutil
import json
from collections import deque
import time
import re
import datetime
import traceback
import numpy as np
from typing import Dict, Optional, List, Tuple, Union, Iterable, Any, Set
import pdb

import torch
import torch.optim.lr_scheduler
from torch.nn.parallel import replicate, parallel_apply
from torch.nn.parallel.scatter_gather import scatter_kwargs, gather
from tensorboardX import SummaryWriter

from itertools import tee

from allennlp.common import Params
from allennlp.common.checks import ConfigurationError
from allennlp.common.util import peak_memory_mb, gpu_memory_mb
from allennlp.common.tqdm import Tqdm
from allennlp.data.instance import Instance
from allennlp.data.iterators.data_iterator import DataIterator
from allennlp.models.model import Model
from allennlp.nn import util
from allennlp.training.learning_rate_schedulers import LearningRateScheduler
from allennlp.training.optimizers import Optimizer

from AttentionSegmentation.commons.trainer_utils import is_sparse,\
    sparse_clip_norm, move_optimizer_to_cuda, TensorboardWriter
# from AttentionSegmentation.visualization.visualize_attns \
#     import html_visualizer
from AttentionSegmentation.model.attn2labels import BasePredictionClass
logger = logging.getLogger(__name__)

TQDM_COLUMNS = 200

import sys
sys.path.insert(0,'/home/ytaille/deep_multilingual_normalization')
from deep_multilingual_normalization.create_classifiers import create_classifiers
from nlstruct.dataloaders import load_from_brat

logger2 = logging.getLogger("nlstruct")
logger2.setLevel(logging.ERROR)

from notebook_utils import *

def _train_epoch(self, epoch: int) -> Dict[str, float]:
        """
        Trains one epoch and returns metrics.
        """
        logger.info(f"Peak CPU memory usage MB: {peak_memory_mb()}")
        if torch.cuda.is_available():
            for gpu, memory in gpu_memory_mb().items():
                logger.info(f"GPU {gpu} memory usage MB: {memory}")

        train_loss = 0.0

        from allennlp.data.fields.array_field import ArrayField

        for i, td in enumerate(self._train_data):
            td.fields['sample_id'] = ArrayField(np.array([i]))

        # Get tqdm for the training batches
        train_generator = self._iterator(self._train_data,
                                         num_epochs=1,
                                         cuda_device=self._iterator_device,
                                         shuffle=True,
                                         )

        train_generator, cp_generator, id_generator = tee(train_generator, 3)

        ids = []

        for ig in id_generator:
            ids.extend([int(sid.item()) for sid in ig['sample_id']])

        shuffled_train_data = [self._train_data[i] for i in ids]

#         train_predictions = self._segmenter.get_predictions(
#                     instances=shuffled_train_data,
#                     iterator = cp_generator,
#                     model=self._model,
#                     cuda_device=self._iterator_device,
#                     verbose=True)

        num_training_batches = self._iterator.get_num_batches(self._train_data)
        train_generator_tqdm = Tqdm.tqdm(train_generator,
                                         total=num_training_batches
                                         )
        self._last_log = time.time()
        last_save_time = time.time()

        batches_this_epoch = 0
        if self._batch_num_total is None:
            self._batch_num_total = 0

        cpt_batch = 0

        # Set the model to "train" mode.
        self._model.train()

    
        for batch in train_generator_tqdm:
            
            batches_this_epoch += 1
            self._batch_num_total += 1
            batch_num_total = self._batch_num_total
            batch_len = len(batch['labels'])

            # FOR train_predictions:
            # pred/gold is sentence level
            # pred_labels/gold_labels is word level


            # FOR batch:
            # labels is sentence level
            # tags is word level

            # print(train_texts)
            # print("SENTENCE LEVEL")
            # print([tp['gold'] for tp in train_predictions[:10]])
            # print(batch['labels'][:10])

            # print("WORD LEVEL")
            # print([tp['gold_labels'] for tp in train_predictions[:2]])
            # print(batch['tags'][:2])

            # exit()
            
            output_dict = self._model(**batch)
            
            attns = output_dict['attentions']
            
            # Policy is "attention mask": attention scores should be higher if we want to predict CUI
            # Only take words with attention above threshold when predicting with deep norm -> see if it's enough (reward indicates that)
            # REINFORCE algo: (also known as Monte Carlo PG)
            # - draw N trajectories (N attention paths?) -> discretise attentions to make them 1 / 0? -> see if it works with bernoulli first
            # - evaluate each trajectory then sum (maybe add baseline -> subtract mean of all trajectories rewards)
            # - Expected return is given by sum(prob(Ti | W) * reward(Ti)) -> see again if it works with bernoulli first
            # W are WeakL weights 
            # - Gradient ascent of return / gradient descent of negative return
            
            # Set horizon ? -> number / proportion of attention at 1 per batch
            # Set number of trajectories ? -> maybe make trajectories number vary based on sentence length
            # gamma = 0.9 ? -> used to simulate temporal importance of reward (multiply each step by a certain power of gamma, furthest rewards are less impactful) -> may not be possible to model here
            
            horizon = 0.2
            n_trajectories = 10
            gamma = 0.9
            attn_threshold = 0.01
            
            mask = batch['tokens']['mask']
            
            # PROBLEM HERE BECAUSE "O" LABEL NOT TAKEN INTO ACCOUNT WITH SOFTMAX -> works with threshold (but without discretizing)
            
#             prob_attn = (attns>attn_threshold).to(float).to('cpu')
#             prob_attn = torch.softmax(attns, -1).to(float).to('cpu')
            prob_attn = attns
            from torch.distributions import Binomial
            
            m = Binomial(probs=prob_attn) # draw more than one sample
            
            trajectory_scores = []
            
            policy_loss = []
            
            # PREDICT ATTENTION SCORES ONE BY ONE TO SIMULATE RL TRAJECTORIES ? -> very heavy computation 
            # use like a density estimation
            
            for traj in range(n_trajectories):
                
                attn_sample = m.sample()
                attn_mask = (attn_sample.sum(-1) > 0) # attn_mask is used to apply to tokens/labels
                
                real_tokens = [np.array(b.fields['tokens'].tokens) for b in shuffled_train_data[cpt_batch:cpt_batch+batch_len]]
#                 gold_labels = [np.array(b.fields['tags'].labels) for b in shuffled_train_data[cpt_batch:cpt_batch+batch_len]]
                gold_norm_labels = [np.array(b.fields['chunk_tags'].labels) for b in shuffled_train_data[cpt_batch:cpt_batch+batch_len]]

                masked_tokens = [rt[attn_mask[w_id,:len(rt)].cpu().to(bool)] if len(rt) > 1 else rt[[attn_mask[w_id,:len(rt)].cpu().to(bool)]] for w_id, rt in enumerate(real_tokens)] # weird behaviour for len == 1
#                 masked_gold = [rt[attn_mask[w_id,:len(rt)].cpu().to(bool)] for w_id, rt in enumerate(gold_labels)]
                masked_gold_norm = [rt[attn_mask[w_id,:len(rt)].cpu().to(bool)] if len(rt) > 1 else rt[[attn_mask[w_id,:len(rt)].cpu().to(bool)]] for w_id, rt in enumerate(gold_norm_labels)]
    
                save_to_ann(masked_tokens, masked_gold_norm, '/home/ytaille/data/tmp/ws_inputs/')
            
# #                 for imgn, mgn in enumerate(masked_gold_norm):
# #                     if not all([m=='O' for m in mgn]):
# #                         print("ID:", imgn)
# #                         print(attn_mask[imgn].sum())
# #                         for rt, gn in zip(real_tokens[imgn],gold_norm_labels[imgn]):
# #                             print(f'{rt}:{gn}')
# #                         print("MASKED",)
# #                         print(masked_tokens[imgn])
# #                         print(masked_gold_norm[imgn])

# #                         raise
                

                # NLSTRUCT PART

                bert_name = "bert-base-multilingual-uncased"

                dataset = load_from_brat("/home/ytaille/data/tmp/ws_inputs")
                dataset['mentions']['mention_id'] = dataset['mentions']['doc_id'] +'.'+ dataset['mentions']['mention_id'].astype(str)

                try:
                    batcher, vocs, mention_ids = preprocess_train(
                        dataset,
                        vocabularies=self.vocabularies1,
                        bert_name=bert_name,
                    )

                    batch_size = len(batcher)
                    with_tqdm = True

                    tg.set_device('cuda:0')
                    device = tg.device

                    pred_batcher = predict(batcher, self.classifier2)

                    scores = compute_scores(pred_batcher, batcher)
                    trajectory_scores.append(scores['loss'] * m.log_prob(attn_sample))
                    

                except:
                    break
                    pass
            
#             def finish_episode():
#                 R = 0
#                 policy_loss = []
#                 returns = []
#                 for r in policy.rewards[::-1]:
#                     R = r + args.gamma * R
#                     returns.insert(0, R)
#                 returns = torch.tensor(returns)
#                 returns = (returns - returns.mean()) / (returns.std() + eps)
#                 for log_prob, R in zip(policy.saved_log_probs, returns):
#                     policy_loss.append(-log_prob * R)
#                 optimizer.zero_grad()
#                 policy_loss = torch.cat(policy_loss).sum()
#                 policy_loss.backward()
#                 optimizer.step()
#                 del policy.rewards[:]
#                 del policy.saved_log_probs[:]
            
            
            cpt_batch += batch_len
            
            
#             if len(policy_loss):
#                 policy_loss = [ts * prob_attn for ts in trajectory_scores]
#                 policy_loss = torch.cat(policy_loss).sum()
#                 print("POLICY loss", policy_loss)
                
            if len(trajectory_scores):
                policy_loss = [ts for ts in trajectory_scores]
                policy_loss = torch.cat(policy_loss).sum()
                print("POLICY loss", policy_loss)
                
            else: policy_loss = 0
                
            self._optimizer.zero_grad()
            loss = self._batch_loss(batch, for_training=True) + policy_loss 
            loss.backward()

            # Make sure Variable is on the cpu before converting to numpy.
            # .cpu() is a no-op if you aren't using GPUs.
            train_loss += loss.data.cpu().numpy()

            batch_grad_norm = self._rescale_gradients()

            # This does nothing if batch_num_total is None or you are using an
            # LRScheduler which doesn't update per batch.
            if self._learning_rate_scheduler:
                self._learning_rate_scheduler.step_batch(batch_num_total)
            self._optimizer.step()

            # Update the description with the latest metrics
            metrics = self._get_metrics(train_loss, batches_this_epoch)
            description = self._description_from_metrics(metrics)

            train_generator_tqdm.set_description(description, refresh=False)
            if hasattr(self, "_tf_params") and self._tf_params is not None:
                # We have TF logging
                if self._batch_num_total % self._tf_params["log_every"] == 0:
                    self._tf_log(metrics, self._batch_num_total)

        return self._get_metrics(train_loss, batches_this_epoch, reset=True)
    
import functools
    
trainer._train_epoch = functools.partial(_train_epoch, trainer)

In [6]:
trainer.train()

2020-12-14 11:34:15,109: INFO: Beginning training.
2020-12-14 11:34:15,111: INFO: ==================================================
2020-12-14 11:34:15,112: INFO: Starting Training Epoch 1/50
2020-12-14 11:34:15,113: INFO: Peak CPU memory usage MB: 6982.376
2020-12-14 11:34:15,208: INFO: GPU 0 memory usage MB: 7358


/home/ytaille/AttentionSegmentation/allennlp/data/fields/array_field.py:42: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return_array[slices] = self.array
  0%|          | 0/57 [00:00<?, ?it/s]

POLICY loss tensor(-112292.7109, device='cuda:0', grad_fn=<SumBackward0>)


CompositeMention: 0.0000, DiseaseClass: 0.2031, Modifier: 0.3281, SpecificDisease: 0.6094, accuracy: 0.2852, loss: -95642.8984 ||:   4%|▎         | 2/57 [00:26<12:13, 13.34s/it] 

POLICY loss tensor(-78994.3594, device='cuda:0', grad_fn=<SumBackward0>)
POLICY loss tensor(-119193.4609, device='cuda:0', grad_fn=<SumBackward0>)


...

POLICY loss tensor(-8482.6709, device='cuda:0', grad_fn=<SumBackward0>)


CompositeMention: 0.9395, DiseaseClass: 0.4248, Modifier: 0.4038, SpecificDisease: 0.6328, accuracy: 0.6002, loss: -6085.1196 ||: 100%|██████████| 57/57 [04:22<00:00,  4.60s/it]

2020-12-14 12:46:14,040: INFO: Starting with Validation



CompositeMention: 0.8433, DiseaseClass: 0.3605, Modifier: 0.4013, SpecificDisease: 0.5705, accuracy: 0.5439, loss: 0.5876 ||: 100%|██████████| 10/10 [00:01<00:00,  7.57it/s]

2020-12-14 12:46:15,365: INFO: Validation done. (959.0 / 1276) zero predicted
2020-12-14 12:46:15,366: INFO: Ran out of patience. Stopping training.


In [ ]:
logger.info("Training Done.")
if instances_test is not None:
    logger.info("Computing final Test Accurje sacy")
    trainer.test(instances_test)
logger.info("Done.")